In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
import jax
import jax.numpy as jnp
import networkx as nx
from datasets.utils import vec_to_graph
from datasets.random_graphs import ErdosRenyiDataset
import pandas as pd
import json 

In [ ]:
#Load the document Experiment1_params.csv
df = pd.read_csv('Experiment1params_simA.csv')
probs, all_params = df['Prob'], df['Final Params'].apply(json.loads)

In [57]:
#Circuito IQP
nqubits = 15
gates = local_gates(n_qubits=nqubits,max_weight=2)


circ = iqp.IqpSimulator(nqubits,gates,device='lightning.qubit')

In [50]:
from pennylane import numpy as pnp 
import pennylane as qml
from qiskit_ibm_runtime import QiskitRuntimeService

token = '146e145beb0b6f91fdd536bf9c4ca6163fa05efb028d71d15dd3e664fcbb4f6350ca716ac1580841936eff98dc60cc38e07e5624ba2cfbc784fad70dfc000cdd'
service = QiskitRuntimeService(channel='ibm_quantum',token=token)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=45)
dev = qml.device('qiskit.remote', wires=15, backend=backend) #shots se ponen aqui


In [ ]:
p = probs[12]
params = all_params[12]
@qml.qnode(dev)
def sample():
    IPQ = circ.iqp_circuit(params)
    return qml.sample(wires = range(15))

In [ ]:
result = sample()
prob = 0
count = 0
for v in result:
    prob += sum(v)/15

print('Mean probability from IQP sampled graphs:', prob/len(result),' for p =', p)

Mean probability from IQP sampled graphs: 0.4722656249999978  for p = 0.35
Ratio of bipartite from IQP sampled graphs: 0.0 for ratio 0.8
